```{contents}
```
## Staging Environment

A **staging environment** in LangGraph is a **pre-production execution environment** that mirrors production as closely as possible and is used to **validate graph behavior, agent logic, data flow, safety controls, and performance** before deployment to real users.

It is the **last line of defense** against failures in live LLM systems.

---

### **1. Why Staging Is Critical for LangGraph**

LangGraph systems are **stateful, non-deterministic, and multi-agent**, which makes failures:

* Hard to reproduce
* Expensive in production
* Dangerous in regulated workflows

Staging provides:

| Guarantee              | Purpose                               |
| ---------------------- | ------------------------------------- |
| Functional correctness | Does the graph do what we designed?   |
| Safety                 | Are destructive actions blocked?      |
| Performance            | Is latency acceptable?                |
| Cost control           | Is token usage stable?                |
| Reliability            | Do retries, loops, and recovery work? |
| Governance             | Are logs and approvals enforced?      |

---

### **2. Environment Architecture**

```
Developer → CI/CD → Staging → Production
                    |
             (Same topology)
```

**Staging mirrors production:**

| Component         | Production         | Staging            |
| ----------------- | ------------------ | ------------------ |
| Graph definitions | Same               | Same               |
| LLM models        | Same family        | Same family        |
| Tools             | Same interfaces    | Mocked or sandbox  |
| State store       | Isolated DB        | Isolated DB        |
| Memory store      | Isolated vector DB | Isolated vector DB |
| Secrets           | Different keys     | Different keys     |
| Users             | Internal           | No customers       |

---

### **3. Staging-Specific Capabilities**

| Capability           | Description                        |
| -------------------- | ---------------------------------- |
| Replay mode          | Re-run executions from checkpoints |
| Forced errors        | Inject failures                    |
| Cost instrumentation | Measure token & tool usage         |
| Human simulation     | Test approvals & overrides         |
| Shadow traffic       | Duplicate real traffic safely      |
| Version testing      | Validate new graph versions        |

---

### **4. Typical LangGraph Staging Workflow**

```
Code Push
   ↓
CI Builds Graph
   ↓
Deploy to Staging
   ↓
Run Synthetic Test Scenarios
   ↓
Execute Agent Simulations
   ↓
Inspect Logs, State, Metrics
   ↓
Human Review & Signoff
   ↓
Promote to Production
```

---

### **5. Example: Staging Configuration**

```python
ENV = "staging"

config = {
    "environment": ENV,
    "recursion_limit": 20,
    "callbacks": [staging_tracer],
    "checkpointing": True,
    "cost_tracking": True,
}
```

```python
result = graph.invoke(input_data, config=config)
```

---

### **6. Staging Safety Controls**

| Control              | Function                  |
| -------------------- | ------------------------- |
| Tool sandbox         | Prevent real-world damage |
| Mock APIs            | Simulate external systems |
| Rate limits          | Prevent runaway loops     |
| Human approval gates | Test oversight            |
| Kill switch          | Emergency stop            |

---

### **7. Testing Dimensions in Staging**

| Dimension        | What You Validate            |
| ---------------- | ---------------------------- |
| Graph logic      | Correct routing, termination |
| Agent behavior   | Correct task delegation      |
| State evolution  | No corruption, no leaks      |
| Memory           | Proper retrieval & storage   |
| Failure handling | Retry, fallback, recovery    |
| Concurrency      | Race condition safety        |
| Scalability      | Load and stress behavior     |

---

### **8. Common Staging Failures Caught Early**

* Infinite reasoning loops
* State corruption across threads
* Runaway tool costs
* Unsafe tool calls
* Agent deadlocks
* Memory leaks
* Improper human approval bypass

---

### **9. Promotion Rule**

A LangGraph system **must never** be promoted unless:

* Every graph version passes staging
* All failure modes are exercised
* Observability is complete
* Cost envelope is approved
* Human oversight works end-to-end

---

### **10. Mental Model**

> **Staging is where LangGraph systems prove they deserve production.**

It converts a fragile LLM prototype into a **reliable autonomous system**.

---

### Demonstration

In [1]:
# ====== STAGING ENVIRONMENT DEMO FOR LANGGRAPH ======

from typing import TypedDict
from langgraph.graph import StateGraph, END

# -----------------------------
# 1. Environment Configuration
# -----------------------------
ENV = "staging"

class State(TypedDict):
    input: str
    step: int
    approved: bool
    output: str

staging_config = {
    "environment": ENV,
    "recursion_limit": 10,
    "checkpointing": True,
    "cost_tracking": True,
    "debug": True
}

# -----------------------------
# 2. Define Graph Nodes
# -----------------------------
def planner(state: State):
    return {"step": state["step"] + 1}

def executor(state: State):
    return {"output": f"Processed: {state['input']} (step {state['step']})"}

def reviewer(state: State):
    return {"approved": state["step"] >= 2}

# -----------------------------
# 3. Build Graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("plan", planner)
builder.add_node("execute", executor)
builder.add_node("review", reviewer)

builder.set_entry_point("plan")
builder.add_edge("plan", "execute")
builder.add_edge("execute", "review")

builder.add_conditional_edges(
    "review",
    lambda s: END if s["approved"] else "plan",
    {
        "plan": "plan",
        END: END
    }
)

graph = builder.compile()

# -----------------------------
# 4. Run in Staging
# -----------------------------
result = graph.invoke(
    {"input": "Quarterly financial report", "step": 0, "approved": False, "output": ""},
    config=staging_config
)

# -----------------------------
# 5. Inspect Result
# -----------------------------
print("ENV:", ENV)
print("Final State:", result)


ENV: staging
Final State: {'input': 'Quarterly financial report', 'step': 2, 'approved': True, 'output': 'Processed: Quarterly financial report (step 2)'}
